In [1]:
%pylab inline
from pymc3 import *
import pymc3

Populating the interactive namespace from numpy and matplotlib


In [2]:
with Model() as model: 
    x = Normal('x')
    n = Normal('n', mu =x, observed=np.array([100.,200,599]))

In [3]:
pymc3.variational.advi( model=model, n=100000)

({'x': array(224.7403587257544)}, {'x': array(-0.6014367249448148)})

In [8]:
np.exp(-.79)

0.45384479528235583